In [1]:
import os
import json
import requests
import azure.core

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [2]:
resource_name = "Phishingdetection_rs"
workspace_name = "Phishingdetection_ws"
subscriptionID = "776851b3-7a01-4304-92cb-70f54ac4dea8" 
aml_compute_target = "phishd-cluster" 
experiment_name= 'phishing_expirement'

In [3]:
# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')

get_workspace error using subscription_id=776851b3-7a01-4304-92cb-70f54ac4dea8, resource_group_name=Phishingdetection_rs, workspace_name=Phishingdetection_ws


Deploying AppInsights with name phishinginsightsabfcd0ab.
Deployed AppInsights with name phishinginsightsabfcd0ab. Took 6.49 seconds.
Deploying KeyVault with name phishingkeyvaultd302b523.
Deploying StorageAccount with name phishingstorage48361a1f9.
Deploying Workspace with name Phishingdetection_ws.
Deployed KeyVault with name phishingkeyvaultd302b523. Took 22.16 seconds.
Deployed StorageAccount with name phishingstorage48361a1f9. Took 23.83 seconds.
Deployed Workspace with name Phishingdetection_ws. Took 22.94 seconds.


In [4]:
Workspace.from_config()


Workspace.create(name='Phishingdetection_ws', subscription_id='776851b3-7a01-4304-92cb-70f54ac4dea8', resource_group='Phishingdetection_rs')

In [5]:
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

creating new compute target : phishd-cluster
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..............
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached now


In [10]:
exp = Experiment(ws,experiment_name)

In [12]:
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="phishingModel.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: phishing_expirement_1745694060_b58f1fcf
Web View: https://ml.azure.com/runs/phishing_expirement_1745694060_b58f1fcf?wsid=/subscriptions/776851b3-7a01-4304-92cb-70f54ac4dea8/resourcegroups/Phishingdetection_rs/workspaces/Phishingdetection_ws&tid=2d3194e3-1654-46bd-bae2-ad37ba11b0ae

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_0904d15
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_0904d15?wsid=/subscriptions/776851b3-7a01-4304-92cb-70f54ac4dea8/resourcegroups/Phishingdetection_rs/workspaces/Phishingdetection_ws&tid=2d3194e3-1654-46bd-bae2-ad37ba11b0ae
2025-04-26T19:07:24: Logging into Docker registry: 4bddbc7f1d734bcfadab61f5cc97c2e3.azurecr.io
2025-04-26T19:07:24: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
2025-04-26T19:07:25: WARNING! Your password will be stored unencrypted in /ro

{'runId': 'phishing_expirement_1745694060_b58f1fcf',
 'target': 'phishd-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-04-26T19:19:33.678009Z',
 'endTimeUtc': '2025-04-26T19:23:15.000088Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'phishd-cluster',
  'ContentSnapshotId': '1d2dac94-9840-436c-b439-d10dd9471428',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'phishingModel.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'phishd-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,

In [13]:
config=ScriptRunConfig(source_directory="./",script="phishingModel.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)

In [17]:

#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./phishingmodel.pkl",model_name="phishingD_ml_model")

Registering model phishingD_ml_model


In [21]:
env.environment_variables


{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [20]:
model = Model(ws,"phishingD_ml_model")

In [22]:
myenv=Environment(name="phishingD-env")
conda_packages = ['numpy']
pip_packages = ['azureml-sdk', 'azureml-defaults', 'scikit-learn', 'pandas', 'joblib']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.8.5')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)

inference_config = InferenceConfig(entry_script='score.py',source_directory='.',environment=myenv)

In [23]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [24]:
# Deploying the model
service = Model.deploy(ws,"phishingdmodel", 
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           #deployment_target=predenv,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

C:\Users\almal\AppData\Local\Temp\ipykernel_27188\491521870.py:2: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,"phishingdmodel",


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-26 22:54:48+03:00 Creating Container Registry if not exists.
2025-04-26 22:54:49+03:00 Building image..
2025-04-26 23:05:18+03:00 Generating deployment configuration.
2025-04-26 23:05:19+03:00 Submitting deployment to compute..
2025-04-26 23:05:25+03:00 Checking the status of deployment phishingdmodel..
2025-04-26 23:07:15+03:00 Checking the status of inference endpoint phishingdmodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://a11f61e4-15b8-4d6c-ac82-0a78b136ef5c.eastus.azurecontainer.io/score


In [25]:
# Prepare your test data
import pandas as pd
import json
test_message = "Your account needs verification. Click www.suspicious-link.com"
test_data = json.dumps({"data": [test_message]})  # As list
prediction = service.run(input_data=test_data)
prediction

'{"results": [{"message": "Your account needs verification. Click www.suspicious-link.com", "prediction": "safe", "probability": 18.055592522943524, "risk": {"level": "high", "color": "red"}}]}'